Lecture 2: Basic Single & Multi-Table SQL
======================

In [1]:
%load_ext sql
%sql sqlite://

'Connected: @None'

Let's create a table, stuff it with data, and query it!

In [2]:
%%sql drop table if exists product;
create table product(
       pname        varchar primary key, -- name of the product
       price        money,               -- price of the product
       category     varchar,             -- category
       manufacturer varchar NOT NULL     -- manufacturer
);
insert into product values('Gizmo', 19.99, 'Gadgets', 'GizmoWorks');
insert into product values('PowerGizmo', 29.99, 'Gadgets', 'GizmoWorks');
insert into product values('SingleTouch', 149.99, 'Photography', 'Canon');
insert into product values('MultiTouch', 203.99, 'Household', 'Hitachi');

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Let's look at the products..

In [3]:
%sql select * from product;

 * sqlite://
Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks
SingleTouch,149.99,Photography,Canon
MultiTouch,203.99,Household,Hitachi


Some *terminology* for SQL.
--------------------------
* The _name_ of the table is product.
* Each row of the table is called a _row_ or a _tuple_. 
* Notice all tuples have the fields or _attributes_.
* The number of rows is called the _cardinality_ while the number of attributes is called the _arity_

Schema Conventions
-----------------
* The schema of product is written as follows:

> product(<u>pname</u>, price, category, manufacturer)

Underlining a set of attributes indicates that they form a _key_.

* In this case, pname is a key. If the product name was only unique for a given manufacturer, we'd write:

> product(<u>pname</u>, price, category, <u>manufacturer</u>)


Tables Explained
----------------
* A tuple = a record
  * Restriction: all attributes are of atomic type
  * There are many atomic data types in SQL engines, look [here](http://www.postgresql.org/docs/9.4/static/datatype.html) for example.


* A table = a (multi)-set of tuples
  * A multiset is like a list…
  * ... but a mutiset is unordered: 
    * no first(), no next(), no last().

# Outline
* Create a database -- done!

* Simple querying -- now!

* Queries with more than one relation -- next!


# Let the (Simple) querying begin! 
* We'll introduce the basics of SQL by example.
* There are many good SQL tutorials on the web, this is intended to get you started.

> SELECT (attributes)<br>
> FROM (one or more tables)<br>
> WHERE (conditions)

This is the simple SELECT-FROM-WHERE (SFW) block. Let's see some examples!

In [4]:
%%sql
SELECT * from Product 
WHERE category='Gadgets' and price > 20.0;

 * sqlite://
Done.


pname,price,category,manufacturer
PowerGizmo,29.99,Gadgets,GizmoWorks


Let's give an example of *projection*, i.e., we only retain some attributes from the query. 

In [5]:
%%sql 
SELECT Pname, Price, Manufacturer
FROM Product;

 * sqlite://
Done.


pname,price,manufacturer
Gizmo,19.99,GizmoWorks
PowerGizmo,29.99,GizmoWorks
SingleTouch,149.99,Canon
MultiTouch,203.99,Hitachi


* The output is *still* a table, and its schema is 
> Answer(pname, price, manufacturer)

* Of course, we can combine selection and projection.

In [6]:
%%sql
SELECT Pname, Price, Manufacturer
FROM Product
WHERE category='Gadgets';

 * sqlite://
Done.


pname,price,manufacturer
Gizmo,19.99,GizmoWorks
PowerGizmo,29.99,GizmoWorks


The output of a query on a table is again a table 
----------------------------------------------
* This is because the query language is *compositional*
* The output of a query really is a table!
* look at this crazy query, what does it ask for?

In [7]:
%%sql
SELECT * FROM Product;

SELECT
    p.manufacturer, p.pname, p.price
FROM 
    (SELECT distinct p0.Manufacturer
     FROM Product p0
     WHERE p0.price < 20.00) cp, -- this is a nested query!
    Product p
WHERE 
    p.manufacturer = cp.manufacturer and p.price > 20.00

 * sqlite://
Done.
Done.


manufacturer,pname,price
GizmoWorks,PowerGizmo,29.99


Details on SQL
--------------

* Some elements are case insensitive (think: program):
  * Same: SELECT  Select  select
  * Same: Product   product
  * Different: ‘Seattle’  ‘seattle’
  

* Constants (single quotes)
  * ‘abc’  - yes
  * “abc” - no


LIKE
====

The LIKE operator is to search strings, perhaps with wildcards. Format is:
    
> SELECT *
> FROM Products
> WHERE pname like '%gizmo%'

* % matches any number of characters
* \_ matches one character
* The like operator is case sensitive


In [8]:
%%sql
SELECT *  FROM product
where pname LIKE '%Gizmo%'

 * sqlite://
Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks


Eliminating Duplicates
---------------------
* duplicates can sometimes be unwelcome or suprising. 
  * Recall tables are _multisets_!

In [9]:
%sql SELECT category from product;

 * sqlite://
Done.


category
Gadgets
Gadgets
Photography
Household


In [10]:
%%sql 
-- easy to remove duplicates, use the distinct keyword
SELECT DISTINCT category from product;

 * sqlite://
Done.


category
Gadgets
Photography
Household


Ordering the results
---------------------
* Sometimes you want the results ordered, let's see some examples!


In [11]:
%%sql
-- sometimes we want to order the results.
-- order by is ascending by default!
SELECT   pname, price, manufacturer
FROM     Product
WHERE    price > 50
ORDER BY  price, pname

 * sqlite://
Done.


pname,price,manufacturer
SingleTouch,149.99,Canon
MultiTouch,203.99,Hitachi


In [12]:
%%sql
-- sometimes we want to order the results.
-- can order like so, each component individually
SELECT   price, manufacturer
FROM     Product
-- the order is "dictionary order" in the clause.
ORDER BY   manufacturer ASC, price DESC

 * sqlite://
Done.


price,manufacturer
149.99,Canon
29.99,GizmoWorks
19.99,GizmoWorks
203.99,Hitachi
